In [18]:
import gradio as gr
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, set_seed
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.chains import LLMChain
from langchain.schema import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os
import fitz  
from langchain.agents import initialize_agent, load_tools

In [3]:
# Load environment variables.
load_dotenv()

# Set the model name for our LLMs.
OPENAI_MODEL = "gpt-3.5-turbo"
# Store the API key in a variable.
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [4]:
llm=ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name=OPENAI_MODEL, temperature=0.3)

In [6]:
# hf = HuggingFacePipeline.from_model_id(
#     model_id="gpt2",
#     task="text-generation",
#     device_map='auto',
#     pipeline_kwargs={"max_new_tokens": 20}
# )

#-----------------------------------------------------------------

# model_id = "gpt2-large"
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# model = AutoModelForCausalLM.from_pretrained(model_id)
# pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map='auto')

# hf = HuggingFacePipeline(pipeline=pipe)

# # Define the query
# query = "What is Godzilla's Power?"

# # Combine the format and query into a prompt
# prompt = format_template.format(query=query)

# # Calculate the maximum context length (including prompt)
# max_context_length = len(tokenizer(prompt)["input_ids"])

# # Set the desired maximum number of new tokens to generate
# max_new_tokens = 200

# # Generate text using the prompt with the specified maximum number of tokens
# output = pipe(prompt, max_length=max_context_length + max_new_tokens, min_length=max_context_length, eos_token_id=tokenizer.eos_token_id, do_sample=False)

# # Extract the generated text and tokens
# generated_text = output[0]["generated_text"]
# generated_tokens = tokenizer(generated_text)["input_ids"]

# # Ensure the generated text does not exceed the max_new_tokens
# if len(generated_tokens) > max_context_length + max_new_tokens:
#     # Truncate the tokens to the desired length
#     truncated_tokens = generated_tokens[max_context_length:max_context_length + max_new_tokens]
#     # Convert the truncated tokens back to text
#     answer = tokenizer.decode(truncated_tokens, skip_special_tokens=True)
# else:
#     # Extract only the answer part from the generated text
#     answer = generated_text.split("-------------------------------------")[-1].strip()

# # Display the answer
# print(answer)

# def pred(info, query, tokens):
#     # Combine the format and query into a prompt
#     prompt = info.format(query=query)

#     # Calculate the maximum context length (including prompt)
#     max_context_length = len(tokenizer(prompt)["input_ids"])

#     # Set the desired maximum number of new tokens to generate
#     max_new_tokens = int(tokens)

#     # Generate text using the prompt with the specified maximum number of tokens
#     output = pipe(prompt, max_length=max_context_length + max_new_tokens, min_length=max_context_length, eos_token_id=tokenizer.eos_token_id, do_sample=False)

#     # Extract the generated text and tokens
#     generated_text = output[0]["generated_text"]
#     generated_tokens = tokenizer(generated_text)["input_ids"]

#     # Ensure the generated text does not exceed the max_new_tokens
#     if len(generated_tokens) > max_context_length + max_new_tokens:
#         # Truncate the tokens to the desired length
#         truncated_tokens = generated_tokens[max_context_length:max_context_length + max_new_tokens]
#         # Convert the truncated tokens back to text
#         answer = tokenizer.decode(truncated_tokens, skip_special_tokens=True)
#     else:
#         # Extract only the answer part from the generated text
#         answer = generated_text.split("\n\n")[-1].strip()

#     return answer

#-----------------------------------------------------------------

# textGen = pipeline("text-generation", model='EleutherAI/gpt-neo-1.3B', framework='pt')
# Godzilla is a fictional giant monster originating from Japanese cinema. It first appeared
# in the 1954 film "Godzilla," directed by Ishirō Honda. Godzilla is often depicted as a
# prehistoric sea monster awakened and empowered by nuclear radiation, symbolizing the
# consequences of atomic weapons. The character has since become a cultural icon,
# appearing in numerous films, television series, and other media.

# prefix = """
# Here are examples between a human and AI. The human provides a question about Godzilla, and
# the AI provides a single sentence with easy to read words The sentence does not have to include the 
# original question. For example:
# """

# examples = [
#     {
#         "query": "Who is godzilla?",
#         "answer": "Godzilla is a fictional giant monster originating from Japanese cinema. It first appeared in the 1954 film 'Godzilla,' directed by Ishirō Honda. Godzilla is often depicted as a prehistoric sea monster awakened and empowered by nuclear radiation, symbolizing the consequences of atomic weapons. The character has since become a cultural icon, appearing in numerous films, television series, and other media."
#     },
#     {
#         "query": "What is Godzilla's Power?",
#         "answer": "Godzilla's powers vary depending on the iteration, but he is typically depicted as having immense size and strength, able to withstand powerful attacks and cause widespread destruction. In some versions, Godzilla can also emit atomic breath, a powerful beam of radiation that he uses to attack his enemies."
#     },
#     {
#         "query": "Who is Godzilla's friend?",
#         "answer": "In various movies and series, Godzilla has been portrayed both as a destructive force and a protector of humanity. Throughout the franchise, Godzilla has formed alliances with other monsters like Mothra, Rodan, and King Kong to combat common foes. Mothra, in particular, is often depicted as a friend and ally of Godzilla, aiding him in battles against powerful adversaries."
#     }
# ]

# format = """
# Human: {query}
# AI: {answer}
# """
# example_template = PromptTemplate(
#     input_variables=["query", "answer"],
#     template=format
# )

# suffix = """
# Human: {query}
# AI: 
# """

# prompt_template = FewShotPromptTemplate(
#     examples=examples,
#     example_prompt=example_template,
#     input_variables=["query"],
#     prefix=prefix,
#     suffix=suffix,
#     example_separator="\n\n"
# )

In [13]:
# Define the format for the template.
format_template = """
Godzilla is a fictional giant monster originating from Japanese cinema. It first appeared
in the 1954 film "Godzilla," directed by Ishirō Honda. Godzilla is often depicted as a
prehistoric sea monster awakened and empowered by nuclear radiation, symbolizing the
consequences of atomic weapons. The character has since become a cultural icon,
appearing in numerous films, television series, and other media.


{query}
"""

# Set up the built-in wikipedia tool.
tools = load_tools(['wikipedia'], llm=llm)

# Initialize the agent.
agent = initialize_agent(tools, agent="chat-zero-shot-react-description", llm=llm)

# Define the query as a string.
query = {"input": "Who is Godzilla"}

# Run the chain using the query as input and print the result.
result = agent.run(query)
print(result)



Godzilla is a fictional monster, or kaiju, that debuted in the eponymous 1954 film. The character has since become an international pop culture icon, appearing in various media such as films, television shows, video games, novels, and comic books. Godzilla is a prehistoric reptilian monster awakened and empowered by nuclear radiation, and has been featured in a total of 38 films within the Godzilla franchise.


In [37]:
# Create a Gradio Blocks instance for structuring the interface
app = gr.Blocks(fill_height=True)

#ChatBot function
def respond(message, chat_history):
    # Set up the built-in wikipedia tool.
    tools = load_tools(['wikipedia'], llm=llm)

    # Initialize the agent.
    agent = initialize_agent(tools, agent="chat-zero-shot-react-description", handle_parsing_errors=True ,llm=llm)
    bot_message = agent.run(message)
    chat_history.append((message, bot_message))
    return "", chat_history



#Forecast bot function
def pdfBot(query, file, file_data):
    text = ""
    try:
        # Open the PDF file
        pdf_document = fitz.open(stream=file_data, filetype="pdf")

        # Iterate over each page in the PDF
        for page_num in range(pdf_document.page_count):
            page = pdf_document[page_num]
            text += page.get_text()

        # Close the PDF document
        pdf_document.close()

    except Exception as e:
        print(f"Error: {e}")

    format_template = f"{text}\n\n{query}"

    print(format_template)

    input_variables = ["query"]

    prompt_template = PromptTemplate(
        input_variables=input_variables,
        template=format_template
    )

    chain = LLMChain(llm=llm, prompt=prompt_template)

    result = chain.invoke({"query": query})

    return result["text"]


# Add tabs to the Gradio Blocks
with app:
    #A chatbot that will be used to talk with our first model we trained
    with gr.Tab("ChatBot"):
        chatbot = gr.Chatbot()
        msg = gr.Textbox(placeholder="Ask Your Question", show_label=False, scale=1)
        clear = gr.ClearButton([msg, chatbot])
        msg.submit(respond, [msg, chatbot], [msg, chatbot])


    with gr.Tab("PDFUploader"):
        gr.File(label="Upload a file:", type='binary'),
        
    #A basic input/output which will take some text and output a prediction that will be used with our second Model
    with gr.Tab("PDF Bot"):
        with gr.Row():

        
            gr.Interface(fn=pdfBot, allow_flagging='never',
                            inputs=[
                            gr.Textbox(lines=1, placeholder="Input", label="Question"),
                            gr.File(label="Upload a file:", type='binary'),
                            #gr.CheckboxGroup(["USA", "Japan", "Canada"], label="Countries", info="Where is your prediction"),
                            ],
                            outputs=gr.Textbox(lines=12,label="Results", show_copy_button=True, interactive=False))


# Launch the app
app.launch()

c:\Users\Lardex\anaconda3\Lib\site-packages\gradio\utils.py:915: UserWarning: Expected 3 arguments for function <function pdfBot at 0x0000016EF4252020>, received 2.
  warnings.warn(
c:\Users\Lardex\anaconda3\Lib\site-packages\gradio\utils.py:919: UserWarning: Expected at least 3 arguments for function <function pdfBot at 0x0000016EF4252020>, received 2.
  warnings.warn(


Running on local URL:  http://127.0.0.1:7879

To create a public link, set `share=True` in `launch()`.
